In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import datasets
import cv2

# Memuat dataset Fashion MNIST
(train_images_mnist, train_labels_mnist), (test_images_mnist, test_labels_mnist) = fashion_mnist.load_data()
train_dataset = datasets.FashionMNIST('path_ke_folder', train=True, download=True)
test_dataset = datasets.FashionMNIST('path_ke_folder', train=False, download=True)
train_label_fashionmnist = train_dataset.targets
test_label_fashionmnist = test_dataset.targets

# Mengambil Subset label dari MNIST
train_label_fashionmnist = train_label_fashionmnist[:35552]
test_label_fashionmnist = test_label_fashionmnist[:8889]

# Path ke folder dataset
dataset_folder = 'images/'

# Path ke file styles.csv
styles_file = 'styles.csv'

# Membaca data dari file styles.csv
data = pd.read_csv(styles_file)

# Membaca gambar dan label dari folder dataset
images = []

# Memuat label ke dalam variabel
label_csv = data['articleType']


# Mengonversi label FashionImage menjadi tipe data string
label_csv = label_csv.astype(str)

# Mengambil Subset Label dari FashionImage
label_csv_train = label_csv[:35552]
label_csv_test = label_csv[3552:]

# Menggabungkan label FashionImages dengan label MNIST
train_label_gabungan = np.concatenate([train_label_fashionmnist, label_csv_train], axis=0)
test_label_gabungan = np.concatenate([test_label_fashionmnist, label_csv_test], axis=0)

# Menggabungkan gambar dari folder 'images' dengan dataset MNIST
image_folder = 'images/'

for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Mengubah gambar menjadi skala abu-abu (grayscale)
    image = cv2.resize(image, (28, 28))  # Menyesuaikan dimensi gambar menjadi 28x28
    image = np.expand_dims(image, axis=-1)  # Menambahkan dimensi kanal (1) pada gambar
    images.append(image)

# Konversi data gambar menjadi array numpy
images_data = np.array(images)


In [7]:
# Memastikan jumlah gambar dan label sama
train_images_mnist = np.expand_dims(train_images_mnist[:35552], axis=-1)
test_images_mnist = np.expand_dims(test_images_mnist[:8889], axis=-1)

# Mengambil Subset gambar dari FashionImage
images_data_train = images_data[:35552]
images_data_test = images_data[35552:]

# Menggabungkan data gambar dari FashionImage dan MNIST
image_data_train = np.concatenate([images_data_train, train_images_mnist], axis=0)
image_data_test = np.concatenate([images_data_test, test_images_mnist], axis=0)

# Mengubah label menjadi angka menggunakan LabelEncoder
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(train_label_gabungan.astype(str))
labels_test = label_encoder.fit_transform(test_label_gabungan.astype(str))
num_classes = len(label_encoder.classes_)

#mengetahui jumlah data
print('jumlah gambar train ', image_data_train.shape)
print('jumlah label train ', train_label_gabungan.shape)
print('jumlah gambar test ', image_data_test.shape)
print('jumlah label test ', test_label_gabungan.shape)


jumlah gambar  (88882, 28, 28, 1)
jumlah label  (88882,)


In [8]:
# Membagi dataset menjadi data training, data test, dan data validasi
x_train, y_train = image_data_train, train_label_gabungan
x_test, x_val, y_test, y_val = train_test_split(image_data_test, test_label_gabungan, test_size=0.5, random_state=42)

# Normalisasi pixel gambar menjadi rentang [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_val = x_val.astype('float32') / 255

# Membangun arsitektur model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
]) 

# Kompilasi model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Pelatihan model
model.fit(x_train, y_train, batch_size=32,
          epochs=10,
          validation_data=(x_val, y_val))

# Evaluasi model pada data test
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


Epoch 1/10
2223/2223 [==============================] - 89s 39ms/step - loss: 3.4849 - accuracy: 0.0982 - val_loss: 3.3331 - val_accuracy: 0.1063
Epoch 2/10
2223/2223 [==============================] - 82s 37ms/step - loss: 3.3539 - accuracy: 0.1074 - val_loss: 3.2885 - val_accuracy: 0.1206
Epoch 3/10
2223/2223 [==============================] - 82s 37ms/step - loss: 3.3123 - accuracy: 0.1116 - val_loss: 3.2671 - val_accuracy: 0.1178
Epoch 4/10
2223/2223 [==============================] - 90s 41ms/step - loss: 3.2872 - accuracy: 0.1148 - val_loss: 3.2512 - val_accuracy: 0.1233
Epoch 5/10
2223/2223 [==============================] - 97s 44ms/step - loss: 3.2673 - accuracy: 0.1194 - val_loss: 3.2478 - val_accuracy: 0.1261
Epoch 6/10
2223/2223 [==============================] - 87s 39ms/step - loss: 3.2519 - accuracy: 0.1218 - val_loss: 3.2443 - val_accuracy: 0.1254
Epoch 7/10
2223/2223 [==============================] - 89s 40ms/step - loss: 3.2380 - accuracy: 0.1233 - val_loss: 3.2365 -

In [ ]:
# Simpan model
model.save('image_classification_model.h5')
